In [ ]:
import numpy as np
import pickle
import glob
import os
import sys
sys.path.append('/home/kvulic/Vulic/cmos_toolbox_w_spike_sorter/')
from statannotations.Annotator import Annotator

from src.cmos_plotter.Firing_metric_plotter import *

import warnings
import matplotlib.cbook

# Filter out the specific font warning
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
# OR more specifically:
warnings.filterwarnings("ignore", message="findfont: Font family .* not found")

In [ ]:
MAIN_PATH = '...'

In [ ]:
with open(os.path.join(MAIN_PATH, '.../validated_results_full_data_w_speed_and_firing.pkl'), 'rb') as f:
    all_data  = pd.read_pickle(f)

In [ ]:
def custom_boxplot(data, x_metric, y_metric, hue_metric, axes, order, labely, name_label, fontsize=16, labelsize=14, position_line=-0.12):
 
    # Plot the strip plot
    sns.boxplot(data=data, x=x_metric, y=y_metric,
                 legend=False, zorder=1, ax=axes, order=order, width=0.4, fliersize=0, showfliers=False, color = 'dimgray')
 
    # Add statistical annotations
    pairs = [
        ("before", "after"),
        ("before", "after_2"),
        ("after", "after_2"),
    ]
 
    print(data.columns)
    annotator = Annotator(axes, pairs, data=data, x=hue_metric, y=y_metric, order=order)
    annotator.configure(test='Kruskal', text_format='star', loc='inside', fontsize=fontsize, hide_non_significant=True)
    print(f"\nAnnotate for metric {y_metric}")
    annotator.apply_and_annotate()
    
    plt.rcParams['xtick.bottom'] = True
    plt.rcParams['ytick.left'] = True
    #plt.rcParams['font.family'] = 'Arial'
    axes.set_ylabel(labely, fontsize=fontsize)
    axes.set_xlabel("", fontsize=fontsize)
    axes.grid(True, axis="y", linestyle=':', alpha=0.7)
    axes.tick_params(axis='both', which='major', labelsize=labelsize, width=1.5, length=8)
    #axes.yaxis.set_major_locator(ticker.MaxNLocator(5))
    
    # Add grouped x-axis labels
    axes.tick_params(axis='both', which='major')
    xticks = axes.get_xticks()  # Positions of "Soft", "Inter", "Stiff", etc.
    axes.set_xticks(xticks)  # Position of ticks
    axes.set_xticklabels(["Before stim.", "0-5 min", "5-10 min"])
    
    # Get relative x-ticks position
    x_min, x_max = axes.get_xlim()
    ticks = [(tick - x_min) / (x_max - x_min) for tick in axes.get_xticks()]
 
    # Add secondary x-axis
    axes.text((ticks[1]+ticks[2])/2, position_line-0.07, "After stimulation", ha="center", va="center", transform=axes.transAxes, fontsize=fontsize, weight="bold")
 
    
    # Add lines under each group
    axes.annotate("", xy=(ticks[1]-0.07, position_line), xytext=(ticks[2]+0.07, position_line),
                xycoords='axes fraction', textcoords='axes fraction',
                arrowprops=dict(arrowstyle="-", color="black", lw=1.5))
    plt.savefig(os.path.join(MAIN_PATH, 'biTE_stimulation/Full_dataset/Figures', f'{y_metric}_{name_label}.png'), dpi=300, bbox_inches='tight')
    plt.savefig(os.path.join(MAIN_PATH, 'biTE_stimulation/Full_dataset/Figures', f'{y_metric}_{name_label}.pdf'), dpi=300, bbox_inches='tight')

In [ ]:
frequencies = ['5_Hz', '10_Hz', '20_Hz', '40_Hz']
for frequency in frequencies:
    filt = 0.0
    # Filter data for the specific frequency and delay
    filt_data = all_data[all_data['syn probability'] > filt]
    filt_data = filt_data[(filt_data['Stimulation_frequency'] == frequency) & (filt_data['Delay'] == '0_ms') ]
    print(filt_data)
    
    # Create a new DataFrame for the boxplot
    new_df = filt_data.copy()
    #new_df['firing_rate'] = pd.concat([filt_data['source_firing_rate'], filt_data['target_firing_rate']], axis=0).reset_index(drop=True)
    #new_df['Status'] = pd.concat([filt_data['Status'], filt_data['Status']], axis=0).reset_index(drop=True)
    #new_df['conduction_speed'] = pd.concat([filt_data['source_speed'], filt_data['target_speed']], axis=0).reset_index(drop=True)
    
    # Create the boxplot
    fig, axes = plt.subplots(figsize=(8, 6))
    custom_boxplot(new_df, 'Status', 'target_firing_rate', 'Status', axes, ['before', 'after', 'after_2'], 'Target Firing Rate [Hz]', frequency+f'_{filt*100}prob', fontsize=16, labelsize=14, position_line=-0.12)

In [ ]:
frequencies = ['5_Hz', '10_Hz', '20_Hz', '40_Hz']
for frequency in frequencies:
    filt = 0.05
    # Filter data for the specific frequency and delay
    filt_data = all_data[all_data['syn probability'] > filt]
    filt_data = filt_data[(filt_data['Stimulation_frequency'] == frequency) & (filt_data['Delay'] == '0_ms') ]
    print(filt_data)
    
    # Create a new DataFrame for the boxplot
    new_df = pd.DataFrame()
    new_df['firing_rate'] = pd.concat([filt_data['source_firing_rate'], filt_data['target_firing_rate']], axis=0).reset_index(drop=True)
    new_df['Status'] = pd.concat([filt_data['Status'], filt_data['Status']], axis=0).reset_index(drop=True)
    new_df['conduction_speed'] = pd.concat([filt_data['source_speed'], filt_data['target_speed']], axis=0).reset_index(drop=True)
    
    # Create the boxplot
    fig, axes = plt.subplots(figsize=(8, 6))
    custom_boxplot(new_df, 'Status', 'conduction_speed', 'Status', axes, ['before', 'after', 'after_2'], 'Conduction speed [m/s]', frequency+f'_{filt*100}prob', fontsize=16, labelsize=14, position_line=-0.12)